# Importe de paquetes

In [1]:
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dir(pyaudio)

['PyAudio',
 'Stream',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__docformat__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__version__',
 'get_format_from_width',
 'get_portaudio_version',
 'get_portaudio_version_text',
 'get_sample_size',
 'pa',
 'paAL',
 'paALSA',
 'paASIO',
 'paAbort',
 'paBadIODeviceCombination',
 'paBadStreamPtr',
 'paBeOS',
 'paBufferTooBig',
 'paBufferTooSmall',
 'paCanNotReadFromACallbackStream',
 'paCanNotReadFromAnOutputOnlyStream',
 'paCanNotWriteToACallbackStream',
 'paCanNotWriteToAnInputOnlyStream',
 'paComplete',
 'paContinue',
 'paCoreAudio',
 'paCustomFormat',
 'paDeviceUnavailable',
 'paDirectSound',
 'paFloat32',
 'paHostApiNotFound',
 'paInDevelopment',
 'paIncompatibleHostApiSpecificStreamInfo',
 'paIncompatibleStreamHostApi',
 'paInputOverflow',
 'paInputOverflowed',
 'paInputUnderflow',
 'paInsufficientMemory',
 'paInt16',
 'paInt24',
 'paInt32',
 'paInt8',
 'paInternalError',
 'paInvalidChannelCou

# Generadores

Los generadores son funciones que conservan su memoria en el momento que se les pide retornar el control al ámbito global (global scope) mediante la palabra clave `yield`.

Esta es diferente a `return` en que `return` invoca al recolector de basura de python para que elimine todas las referencias y los objetos de variables utilizadas dentro del ámbito de la función, mientras que `yield` las preserva para cuando se le otorgue el control de nuevo a la función.

In [3]:
def prueba_generador():
    i = 1
    while True:     # Un ciclo infinito
        yield(i)    # Se retorna el control al ámbito global junto al valor de i
        i = i + 1   # Se modificará el valor de "i" POSTERIOR a recobrar el control

Se inicializa el generador

In [4]:
generador = prueba_generador()

La palabra clave `next` se utiliza para movernos sobre los objetos de un iterador

In [10]:
next(generador)

6

**¡Ojo!:** Los objetos iterables como listas, tuplas, etc. que hemos visto NO son iteradores por defecto, si no que necesitan ser convertidos primero:

In [11]:
lista_prueba = [1,4,3,2,6]

In [12]:
next(lista_prueba)

TypeError: 'list' object is not an iterator

In [13]:
iterador_prueba = iter(lista_prueba)

In [19]:
next(iterador_prueba)

StopIteration: 

In [20]:
def help_getter(modulo):
    directorio = [func for func in dir(modulo) if not func.startswith("_")]
    directorio = iter(directorio)  # Iterador de funciones en el módulo
    while directorio:  # Mientras no lo recorramos todo...
        func = next(directorio)  # Obtenemos la siguiente función en el directorio
        exec(f"print(help(pyaudio.{func}))") # Ejecutamos help() para dicha función
        yield   # Retornamos el control hasta ser llamada de nuevo.

Inicializando:

In [21]:
generador_ayuda = help_getter(pyaudio)

In [29]:
next(generador_ayuda)

Help on int object:

class int(object)
 |  int([x]) -> integer
 |  int(x, base=10) -> integer
 |  
 |  Convert a number or string to an integer, or return 0 if no arguments
 |  are given.  If x is a number, return x.__int__().  For floating point
 |  numbers, this truncates towards zero.
 |  
 |  If x is not a number or if base is given, then x must be a string,
 |  bytes, or bytearray instance representing an integer literal in the
 |  given base.  The literal can be preceded by '+' or '-' and be surrounded
 |  by whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.
 |  Base 0 means to interpret the base from the string as an integer literal.
 |  >>> int('0b100', base=0)
 |  4
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __ceil__(...)
 |      Ceiling of an Integral retur

# Configuración básica de un canal de entrada

In [30]:
import os
import time
from tkinter import TclError

In [31]:
p = pyaudio.PyAudio()

In [32]:
p.get_default_input_device_info()

{'index': 8,
 'structVersion': 2,
 'name': 'default',
 'hostApi': 0,
 'maxInputChannels': 32,
 'maxOutputChannels': 32,
 'defaultLowInputLatency': 0.008684807256235827,
 'defaultLowOutputLatency': 0.008684807256235827,
 'defaultHighInputLatency': 0.034807256235827665,
 'defaultHighOutputLatency': 0.034807256235827665,
 'defaultSampleRate': 44100.0}

In [38]:
p.get_device_info_by_index(5)

{'index': 5,
 'structVersion': 2,
 'name': 'HDA NVidia: HDMI 0 (hw:1,3)',
 'hostApi': 0,
 'maxInputChannels': 0,
 'maxOutputChannels': 2,
 'defaultLowInputLatency': -1.0,
 'defaultLowOutputLatency': 0.005804988662131519,
 'defaultHighInputLatency': -1.0,
 'defaultHighOutputLatency': 0.034829931972789115,
 'defaultSampleRate': 44100.0}

In [ ]:
help(pyaudio.Stream.__init__)

Necesitamos explicar algunos conceptos:
* Buffer: Región reservada de memoria donde temporalmente se almacenará algo pero se sobreescribe en demanda de nueva data a guardar

* Chunks: Número de muestras a guardar en el buffer a la vez. En el ejemplo de abajo, al elegir 2048 como tamaño del chunk, signfica que tendremos 2048 muestras antes de que nuestro buffer se llene y debamos sobreescribir la información. Esta información es almacenada para poder procesarla adecuadamente (ej. algoritmos para discernir si la señal es asemejada a un ruido blanco o catalogable como voz humana) y evitar fugas de memoria

* Format: El tipo de dato interno en memoria en el cual guardaremos **cada elemento de la muestra**. Esto es conocido como "[bit depth](https://en.wikipedia.org/wiki/Audio_bit_depth)" y define cuántos posibles niveles distinguibles de "volumen" podremos representar digitalmente. El estándar de la industria para productos finales es de 16 pero se puede llegar a necesitar 24 en casos de grabaciones individuales de instrumentos con partes tenues (como un baterista tocando el platillo con suavidad) que quieran ser capturadas sin distorción.

* Channel: Es un canal para flujo continuo de información unidimensional, si agregamos más canales (no todos los dispositivos pueden) podríamos obtener más de una cadena continua de información y, por ejemplo, triangular la posición de la fuente de sonido.

* Rate: Tasa de muestreo (muestras por segundo). Es decir, cada segundo que pasa, en este caso, se toman 44,100 puntos. Esto es porque la frecuencia que necesitamos digitalizar debe ser reconstruirse a un mínimo de precisión de 20 kHz (el máximo estimado de audición humana). 

  Para lograr esta precisión, según el [teorema de Nyquist-Shannon](https://en.wikipedia.org/wiki/Nyquist%E2%80%93Shannon_sampling_theorem), necesitamos al menos tantos puntos por segundo en nuestra señal discreta (digital) como el doble de la frecuencia de oscilación de la señal continua (análoga). Esto quiere decir que ocupamos una tasa de muetreo de 40 kHz, o bien, 40,000 puntos por segundo.
  
  Se adiciona un poco más ([44,100](https://en.wikipedia.org/wiki/44,100_Hz)) por múltiples razones. Quizá la más importante siendo el evitar [solapamiento/aliasing](https://en.wikipedia.org/wiki/Aliasing) que igual puede ocurrir por pequeños defectos en múltiples etapas del proceso de almacenamiento y restauración del audio o ruido dentro de la señal. Para eso se utiliza un filtro [anti-solapamiento](https://en.wikipedia.org/wiki/Anti-aliasing_filter) que en la práctica necesita de una [banda de transición](https://en.wikipedia.org/wiki/Transition_band) y en este caso nos podemos permitir una banda de transición de 2.05 kHz.


In [39]:
# Se utiliza para que las figuras de matplotlib se abran en otra ventana aparte
%matplotlib tk

# constantes
CHUNK = 1024 * 2                 # número de objetos (de tipo FORMAT) almacenados a la vez
FORMAT = pyaudio.paInt16     # Tipo de dato de cada observación
CHANNELS = 1                 # Un canal para el micrófono
RATE = 44100                 # muestras por segundo, i.e 44.1 kHz, estándar de la industria 
                             # para CDs, mp3. También 48 kHz es común para DVDs y Blu-rays
# input_device_index=13

In [41]:
# Creando figura de matplotlib
fig, ax = plt.subplots(1, figsize=(15, 7))

# Instancia de clase PyAudio para administrar los canales de entrada y salida
p = pyaudio.PyAudio()

# Se abre el flujo (stream) para la transmisión de data en tiempo real
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=False,
    frames_per_buffer=CHUNK
)

"""
Como tendremos en memoria tantos puntos como el chunk size nos permite, utilizaremos CHUNK número de puntos en el dominio (x). 
Tenemos tamaño de paso igual a 2 porque la data se nos otorga en bytes (8 bits) y nuestro tipo de dato utilizado es Int16, que 
se almacena utilizando 16 bits. Por ello, cada elemento de la muestra tomada contiene 2 bytes.
"""
x = np.arange(0, 2 * CHUNK, 2) # x = [0, 2, 4, ..., 4090, 4092]

# Creamos un objeto linea (al azar, pues no importa su valor inicial), el cual se actualizará dentro de un ciclo de repetición.
# Esto evita el necesitar actualizar la figura entera del pyplot, lo cual sería demasiado lento.
line, = ax.plot(x, np.random.rand(CHUNK), '-', lw=2)

# Formato del objeto Ejes
ax.set_title('Audio Waveform')
ax.set_xlabel('Muestras')
ax.set_ylabel('Volumen')
ax.set_ylim(-2**15, 2**15-1)

# Incializa la figura
plt.show(block=False)

print('Canal abierto')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    # Leemos los datos binarios que vienen en el flujo
    data = stream.read(CHUNK)  # data = [1,0,0,1,0,1,1,1,...] 
    
    # Como la data la queremos en Int16, usamos 'h' al final.
    # en caso de haber querido bytes, se utiliza una 'B '
    data_np = struct.unpack(str(CHUNK) + 'h', data)

    # Se actualiza la linea
    line.set_ydata(data_np)
    
    # Se actualiza la figura en el canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
    
    # Entramos aquí al cerrar la figura y por ende recibir un TclError
    except TclError:
        
        # Cerramos el canal de audio
        p.close(stream)
        
        # Posteriormente, calcula el FPS
        frame_rate = frame_count / (time.time() - start_time)
        
        print('Canal cerrado')
        print('FPS promedio = {:.0f} FPS'.format(frame_rate))
        break

Canal abierto
Canal cerrado
FPS promedio = 21 FPS


# Análisis del espectro de frecuencias 

In [42]:
# Necesitaremos fft: La transformada rápida de fourier
from scipy.fftpack import fft

In [43]:
# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=False,
    frames_per_buffer=CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)

# create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)

# create semilogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# format waveform axes
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(-2**15, 2**15-1)
#ax1.set_ylim(0, 255)
#ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK])

# format spectrum axes
ax2.set_xlim(20, RATE / 2)
ax2.set_ylim(0, 25)
ax2.set_xlabel('Frecuencias')

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(CHUNK)  
    
    # convert data to integers, make np array, then offset it by 127
    #data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    #data_np = np.array(data_int, dtype='b')[::2] + 128
    data_np = struct.unpack(str(CHUNK) + 'h', data)
    
    
    line.set_ydata(data_np)
    
    # compute FFT and update line
    yf = fft(data_np)
    line_fft.set_ydata(np.abs(yf[0:CHUNK])  / (128 * CHUNK))
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        # Cerramos el canal de audio
        p.close(stream)
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break

stream started
stream stopped
average frame rate = 21 FPS
